In [1]:
import sys, os, re, copy
import dill as pickle # this serializes all the functions inside the quantification dict
import numpy as np
import scipy as sp
from scipy.optimize import newton, minimize, fsolve
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import matplotlib.colors as mcolors
import matplotlib.ticker as tck
from matplotlib.gridspec import GridSpec
palette = list(mcolors.TABLEAU_COLORS.keys())
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid", {
 'axes.spines.bottom': True,
 'axes.spines.left': True,
 'axes.spines.right': True,
 'axes.spines.top': True
})
sns.set(font_scale=1)
palette = list(mcolors.TABLEAU_COLORS.keys())
sns.set_theme(style="ticks", palette="muted")
sns.set_context("talk")

In [2]:
def impute_conc(piece_wise_fit_metab, response_ratio):
    '''
    This function imputes the concentration from a response ratio.
    '''
    response_ratio_range = np.array(list(piece_wise_fit_metab.keys()))
    mask_range = [response_ratio >= min_v and response_ratio <= max_v for max_v, min_v in response_ratio_range]
    k = tuple(response_ratio_range[mask_range][0])
    conc = piece_wise_fit_metab[k](response_ratio)
    estimator = 'intrapolation'
    if 0 in k:
        estimator = 'extrapolation under'
    elif np.inf in k:
        estimator = 'extrapolation over'
    return(conc, estimator)

In [3]:
### Read quantification function ###
dict_pickle_fnam = 'AA_quant-dict.pickle'
with open(dict_pickle_fnam, 'rb') as handle:
    piece_wise_fit_quant = pickle.load(handle)

In [4]:
### Read measurements ###
### Replace all N/F with 0 before start ###
# esheet_dict_mes = pd.read_excel('20230208_Cholangio_media_consumption_analysis.xlsx', sheet_name=None)
esheet_dict_mes = pd.read_excel('20230425_Cholangio_NRF2on_media_consumption_2.xlsx', sheet_name=None)
# annotation_df = pd.read_excel('input/annotations.xlsx')
metab_dict_mes = dict()
metab_names_mes = list()
for k in esheet_dict_mes.keys():
    if 'U-13C' not in k:
        metab_names_mes.append(k)
        metab_dict_mes[k] = copy.deepcopy(esheet_dict_mes[k])
        metab_dict_mes[k]['Response Ratio'] = metab_dict_mes[k]['Area'] / metab_dict_mes[k]['ISTD Response']
        metab_dict_mes[k]['Response Ratio'] = metab_dict_mes[k]['Response Ratio'].fillna(0).replace(np.inf, 0)
        metab_dict_mes[k]['Sample_name'] = [fn.split('_')[-1] for fn in metab_dict_mes[k]['Filename']]
        # Add annotations:
        # metab_dict_mes[k] = metab_dict_mes[k].merge(annotation_df, left_on='Sample_name', right_on='Sample_name')
        metab_dict_mes[k] = metab_dict_mes[k].drop(['Flag Details', 'Filename', 'Type', 'RT'], axis=1)

In [5]:
### Impute concentration and add to metabolite dataframe ###
rr_mes = dict() # for plotting 
imp_conc_mes = dict() # for plotting
for metab in metab_names_mes[:]:
    # Assign imputed concentration:
    if metab in piece_wise_fit_quant:
        conc_list = list()
        estimator_list = list()
        for rr in metab_dict_mes[metab]['Response Ratio'].values:
            conc, estimator = impute_conc(piece_wise_fit_quant[metab], rr)
            conc_list.append(conc)
            estimator_list.append(estimator)
        metab_dict_mes[metab]['imputed_sample_conc'] = conc_list
        metab_dict_mes[metab]['imputed_sample_estimator'] = estimator_list

# metab_dict_mes['Lactate neg']['imputed_sample_conc'] = metab_dict_mes['Lactate neg']['Response Ratio'].values * 144.88
# metab_dict_mes['Lactate neg']['imputed_sample_estimator'] = 'spike-in extrapolation'

# metab_dict_mes['Glucose neg']['imputed_sample_conc'] = metab_dict_mes['Glucose neg']['Response Ratio'].values * 500
# metab_dict_mes['Glucose neg']['imputed_sample_estimator'] = 'spike-in extrapolation'

In [6]:
# Generate dataframe for concentrations and estimtors:
conc_df = metab_dict_mes['Glutamine pos'].loc[:, ['Sample_name']]
est_df = metab_dict_mes['Glutamine pos'].loc[:, ['Sample_name']]

for metab_nam in metab_dict_mes:
    if 'imputed_sample_conc' in metab_dict_mes[metab_nam].columns:
        conc_df = conc_df.merge(metab_dict_mes[metab_nam].loc[:, ['Sample_name', 'imputed_sample_conc']], on='Sample_name')
        conc_df = conc_df.rename(columns={'imputed_sample_conc': metab_nam})
        est_df = est_df.merge(metab_dict_mes[metab_nam].loc[:, ['Sample_name', 'imputed_sample_estimator']], on='Sample_name')
        est_df = est_df.rename(columns={'imputed_sample_estimator': metab_nam})

# Write to Excel file:
# with pd.ExcelWriter('20230208_Cholangio_media_consumption_analysis_conc.xlsx') as writer:
with pd.ExcelWriter('20230425_Cholangio_NRF2on_media_consumption_analysis_conc.xlsx') as writer:
    conc_df.to_excel(writer, sheet_name='conc_uM')
    est_df.to_excel(writer, sheet_name='estimator')